In [1]:
import pickle
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
from utility.albumentations_helper import create_transform
from utility.others import clear_all
import time
from functions import DataGenerator,PredictGenerator,create_model2,dot_distance,exp_loss,\
                        sigmoid_dot_distance,dot_sigmoid_distance,cross_entropy_loss,l2_distance,\
                        margin_loss_fun_factory,Xception_reduced,dot_distance_neg
from utility.grayscale_imagenet import Xception_greyscale
from utility.grayscale_DenseNet import DenseNet_greyscale
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
#from tensorflow.keras.applications import DenseNet121

In [2]:
color = False
shapes = (224,224,1)
HalfBatch = 6

### set up generator

In [13]:
if color:
    with open('/home/will/Desktop/kaggle/Whale/train_df_color.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train_color.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df_color.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val_color.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)
else:
    with open('/home/will/Desktop/kaggle/Whale/train_df.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)

In [14]:
aug = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1),Cutout(p=0.5)])
transform = create_transform(aug)  

aug_test = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1)])
transform_test = create_transform(aug_test)    

gen_train = DataGenerator(Ids_train,newWhale_train,transform,color=color,HalfBatch=HalfBatch)
gen_val = DataGenerator(Ids_val,newWhale_val,transform_test,color=color,HalfBatch=HalfBatch)

### build model

In [5]:
conv_base = DenseNet_greyscale(121,(224,224,1),'max',False)
#conv_base = Xception_greyscale((256,256,1),'max',False)
# conv_base = Xception_reduced((256,256,1))

In [6]:
train_model,feature_model = create_model2(1e-3,margin_loss_fun_factory(0.1,5),conv_base,color)

### training

In [7]:
# conv_base = DenseNet_greyscale(121,(224,224,1),'max',False)
# train_model,feature_model = create_model2(1e-3,margin_loss_fun_factory(0.1,5),conv_base,color)
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
999/999 [==============================] - 84s 84ms/step - loss: -0.1562 - val_loss: -0.4259
Epoch 2/5
999/999 [==============================] - 75s 75ms/step - loss: -0.4896 - val_loss: -0.8072
Epoch 3/5
999/999 [==============================] - 75s 75ms/step - loss: -0.7536 - val_loss: -0.8513
Epoch 4/5
999/999 [==============================] - 76s 76ms/step - loss: -0.9014 - val_loss: -0.9146
Epoch 5/5
999/999 [==============================] - 128s 128ms/step - loss: -0.9894 - val_loss: -0.8713
time:437.3666067123413


In [8]:
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=2,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/2
999/999 [==============================] - 130s 130ms/step - loss: -1.0284 - val_loss: -0.9232
Epoch 2/2
999/999 [==============================] - 131s 131ms/step - loss: -1.0763 - val_loss: -0.9413
time:260.257266998291


In [9]:
train_model.save('Models/DenseNet_Depthwise0.h5')
feature_model.save('Models/feature_model_DenseNet_Depthwise0.h5')

Fine-Tune

In [14]:
feature_model.trainable=True

In [15]:
train = False
for layer in feature_model.layers:
    if 'conv5' in layer.name:
        train = True
    layer.trainable = train

In [17]:
train_model.compile(loss=margin_loss_fun_factory(0.1,5),optimizer=tf.keras.optimizers.Adam(lr=5e-4))

In [18]:
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
999/999 [==============================] - 170s 171ms/step - loss: -1.6267 - val_loss: -1.4251
Epoch 2/5
999/999 [==============================] - 157s 157ms/step - loss: -1.8286 - val_loss: -1.3350
Epoch 3/5
999/999 [==============================] - 157s 157ms/step - loss: -1.8668 - val_loss: -0.8694
Epoch 4/5
999/999 [==============================] - 157s 157ms/step - loss: -1.8900 - val_loss: -1.2918
Epoch 5/5
999/999 [==============================] - 158s 158ms/step - loss: -1.9470 - val_loss: -1.3719
time:804.5997478961945


In [20]:
train_model.save('Models/DenseNet_Depthwise1.h5')
feature_model.save('Models/feature_model_DenseNet_Depthwise1.h5')

In [21]:
train = False
for layer in feature_model.layers:
    if 'conv4' in layer.name:
        train = True
    layer.trainable = train

In [22]:
train_model.compile(loss=margin_loss_fun_factory(0.1,5),optimizer=tf.keras.optimizers.Adam(lr=2e-4))

In [24]:
start = time.time()
history = train_model.fit_generator(
          gen_train,
          validation_data = gen_val,
          epochs=5,
          use_multiprocessing=True,workers=2,max_queue_size=20)
end = time.time()
print('time:{}'.format(end - start))

Epoch 1/5
999/999 [==============================] - 270s 270ms/step - loss: -1.9454 - val_loss: -1.4605
Epoch 2/5
999/999 [==============================] - 243s 243ms/step - loss: -2.0044 - val_loss: -1.6130
Epoch 3/5
999/999 [==============================] - 242s 242ms/step - loss: -2.0701 - val_loss: -1.6065
Epoch 4/5
999/999 [==============================] - 241s 241ms/step - loss: -2.0950 - val_loss: -1.6978
Epoch 5/5
999/999 [==============================] - 242s 243ms/step - loss: -2.1101 - val_loss: -1.7125
time:1251.197096824646


In [25]:
train_model.save('Models/DenseNet_Depthwise2.h5')
feature_model.save('Models/feature_model_DenseNet_Depthwise2.h5')

In [12]:
feature_model = load_model('Models/feature_model_DenseNet_Depthwise2.h5')

In [8]:
conv_weight = np.squeeze(train_model.layers[-2].get_weights()[0])
np.save('Models/conv_weight_Depthwise2.npy',conv_weight)